In [1]:
#   ▄████████    ▄████████ ████████▄  ████████▄   ▄█      ███             ▄████████    ▄████████  ▄████████    ▄█    █▄     ▄█   ▄█    █▄     ▄████████    ▄████████
#   ███    ███   ███    ███ ███   ▀███ ███   ▀███ ███  ▀█████████▄        ███    ███   ███    ███ ███    ███   ███    ███   ███  ███    ███   ███    ███   ███    ███
#   ███    ███   ███    █▀  ███    ███ ███    ███ ███▌    ▀███▀▀██        ███    ███   ███    ███ ███    █▀    ███    ███   ███▌ ███    ███   ███    █▀    ███    ███
#  ▄███▄▄▄▄██▀  ▄███▄▄▄     ███    ███ ███    ███ ███▌     ███   ▀        ███    ███  ▄███▄▄▄▄██▀ ███         ▄███▄▄▄▄███▄▄ ███▌ ███    ███  ▄███▄▄▄      ▄███▄▄▄▄██▀
# ▀▀███▀▀▀▀▀   ▀▀███▀▀▀     ███    ███ ███    ███ ███▌     ███          ▀███████████ ▀▀███▀▀▀▀▀   ███        ▀▀███▀▀▀▀███▀  ███▌ ███    ███ ▀▀███▀▀▀     ▀▀███▀▀▀▀▀
# ▀███████████   ███    █▄  ███    ███ ███    ███ ███      ███            ███    ███ ▀███████████ ███    █▄    ███    ███   ███  ███    ███   ███    █▄  ▀███████████
#   ███    ███   ███    ███ ███   ▄███ ███   ▄███ ███      ███            ███    ███   ███    ███ ███    ███   ███    ███   ███  ███    ███   ███    ███   ███    ███
#   ███    ███   ██████████ ████████▀  ████████▀  █▀      ▄████▀          ███    █▀    ███    ███ ████████▀    ███    █▀    █▀    ▀██████▀    ██████████   ███    ███
#   ███    ███                                                                         ███    ███                                                          ███    ███

# Reddit Archiver - An easy way to archive and view your Reddit Account. 

Downloads your profile info and posts/comments from your input reddit account. Creates a presentable webpage of your arhcived profile that can be saved locally. 

* Allows user to input desired Reddit Username and output directory for archive file via command line
* Downloads user data using BeautifulSoup.
* Saves data to JSON file
* Loads JSON File into HTML webpage and saves webpage to desired directory


By [dandex200](https://github.com/dandex200/) on Github


#### Imports

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import os
from tkinter import filedialog, Tk

## User Input

In [2]:
#Username Input
user_input = input("Enter desired Reddit Username to Archive:")

#Output Directory Input using Tkinter
confirm_input = input("Choose Archive File Output Directory? (Y/N)")

#Select Output Directory if chosen
if confirm_input.lower() == 'y':
    root = Tk() 
    root.withdraw()
    output_dir = filedialog.askdirectory()
    root.destroy()
    print("Will save archive file to: %s" % output_dir)
else:
    print("No output directory selected - will save to current directory.")
    

Will save archive file to: /home/dandex/programming/The_Reddit_Archiver/Reddit_Archiver


## Scraping

In [36]:
user_url = 'https://old.reddit.com/user/'+user_input

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

response = requests.get(user_url, headers=headers)

soup = BeautifulSoup(response.content, 'html.parser')

title_check = soup.title.text.split(' ')[0]

if title_check == "overview":
    print("working")
else:
    print("User does not exist")

working


### Profile Info

In [68]:
username = soup.title.text.split(' ')[2]

comment_karma_box = soup.find('span', {'class': 'comment-karma'})
comment_karma = comment_karma_box.text if comment_karma_box is not None else ''

post_karma = soup.find('span', {'class': 'karma'})
post_karma = post_karma.text if post_karma is not None else ''


user_creation_date = soup.find('span', {'class': 'age'})
user_creation_date = user_creation_date.find('time')['datetime']

num_posts = 0
num_comments = 0

print(username)
print(comment_karma)
print(post_karma)
print(user_creation_date)
print(num_posts)
print(num_comments)

NPU-F
14,754
27,122
2015-11-16T22:40:48+00:00
0
0


### User Posts & Comments

In [ ]:
# JSON FORMAT:
#     Username:
#         Post Karma:
#         Comment Karma:
#         Account Creation Date:
#         Number of Submissions:
#         Number of Comments:
#     Posts:
#         utc_date:
#         Type:
#         Subreddit:
#         Title:
#         File_Path:
#         Permalink:
#         Score:
#     Comments:
#         utc_date:
#         Subreddit:
#         Text:
#         Score:
#         Permalink:
    

In [46]:
comments = []
for comment in soup.find_all('div', class_='comment'):
    utc_date = comment.find('time', class_="live-timestamp")['datetime']
    score = comment.find('span', class_='score').text
    subreddit = comment.find('a', class_="subreddit").text
    permalink = comment.find('a', class_="bylink")['href']
    text = comment.find('div', class_='md-container').text
    comment_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'text': text
    }
    comments.append(comment_data)
print(comments)

[{'utc_date': '2023-04-25T12:48:08+00:00', 'score': '5 points', 'subreddit': 'Atlanta', 'permalink': 'https://old.reddit.com/r/Atlanta/comments/12yh1ox/fulton_board_to_consider_45m_tax_break_for_data/jhn9ei3/', 'text': 'The BeltLine is already a paved trail there. A portion of the west side trail runs from Huff Road south past the jail and Westside Park almost to Donald Lee Hollowell Pkwy.\n\n'}, {'utc_date': '2023-04-25T12:06:54+00:00', 'score': '10 points', 'subreddit': 'Atlanta', 'permalink': 'https://old.reddit.com/r/Atlanta/comments/12yh1ox/fulton_board_to_consider_45m_tax_break_for_data/jhn4rla/', 'text': 'DAFC Zoom Meeting at 2:00pm.\nMeeting Agenda\n\n'}, {'utc_date': '2023-04-21T13:34:00+00:00', 'score': '33 points', 'subreddit': 'Atlanta', 'permalink': 'https://old.reddit.com/r/Atlanta/comments/12tr764/theft_of_atl_airport_funds_by_airport_executives/jh54ypr/', 'text': 'Keisha Lance Bottoms made up a job at the airport for her campaign manager\n\nMarva Lewis, Bottoms’ campaig

In [60]:
posts = []
for post in soup.find_all('div', class_='link'):
    print(post)
    #

<div class="thing id-t3_12yh1ox linkflair linkflair-politics odd link" data-author="NPU-F" data-author-fullname="t2_s3ulw" data-comments-count="28" data-context="listing" data-domain="ajc.com" data-fullname="t3_12yh1ox" data-gildings="0" data-is-gallery="false" data-nsfw="false" data-num-crossposts="0" data-oc="false" data-permalink="/r/Atlanta/comments/12yh1ox/fulton_board_to_consider_45m_tax_break_for_data/" data-promoted="false" data-rank="" data-score="57" data-spoiler="false" data-subreddit="Atlanta" data-subreddit-fullname="t5_2qiq9" data-subreddit-prefixed="r/Atlanta" data-subreddit-type="public" data-timestamp="1682423410000" data-type="link" data-url="https://www.ajc.com/news/fulton-board-to-consider-45m-tax-break-for-data-center-along-beltline/WHGN6YCV7RG2LCGL7ZSWROPZSM/" data-whitelist-status="all_ads" id="thing_t3_12yh1ox" onclick="click_thing(this)"><p class="parent"></p><span class="rank"></span><div class="midcol unvoted"><div aria-label="upvote" class="arrow up login-re

In [ ]:
user_data = {
    'user_info': {
        'username': username,
        'user_creation_date': user_creation_date,
        'num_posts': num_posts,
        'num_comments': num_comments,
        'karma': {
            'post_karma': post_karma,
            'comment_karma': comment_karma
        }
    },
    'posts': posts,
    'comments': comments
}

In [ ]:
with open('Temp_Files/user_data.json', 'w') as f:
    json.dump(user_data, f)

**To-DO: PT1**
* Successfully append comments, user info, and posts of first page to JSON in correct format
* Save pictures/videos to directory
* Present JSON and saved files in beginner format HTML webpage
* Apply saving to all pages of reddit account
* Add tests to ensure data is all saved (meets 25 posts per page, etc)
* Create Readme on Github
* Publish first time

**TO-DO PT2**
* Convert from notebook to script
* Create GUI where you can choose if you want to save pics/videos, input user/output directory in GUI, etc
* Apply ability to save pics/videos as choice
* Create Pretty looking HTML archive page
* Promote on Reddit
* Release as second update 
* Add ability to keep link in comment

**TO-DO PT3**
* 